In [17]:
from pymodbus.client import ModbusSerialClient
from pymodbus.framer import ModbusRtuFramer

The following cell scans all available baudrates and requestst the device address from the modbus broadcast `0x00`.
Just adjust the `port` to your needs.

In [18]:
port = "/dev/ttyUSB1"

baudrates = [
    1200,
    2400,
    4800,
    9600,
    19200
]

scanned_baudrate = 0
scanned_address = 1

for baudrate in baudrates:
    client = ModbusSerialClient(port=port,
                                framer=ModbusRtuFramer,
                                baudrate=baudrate,
                                bytesize=8,
                                parity='N',
                                stopbits=1)
    
    client.connect()
    
    if client.connected:
        print(f"Scanning current Baudrate {baudrate}", end='\r')
        response = client.read_holding_registers(0x0002, 1, slave=0xff)
        if not response.isError():
            print(f"\nCurrent baudrate might be: {baudrate}")
            print(f"Device address might be: {response.registers[0]}")
            scanned_baudrate = baudrate
            client.close()
            break
        else:
            continue
            
client.close()

Scanning current Baudrate 19200
Current baudrate might be: 19200
Device address might be: 5


The following code cell now uses the determined baudrate to scan for slave devices on the bus.
Usually this should work, but sometimes you might have to overwrite the `scanned_baudrate` if the
previous code cell did not work.

In [ ]:
scanner = ModbusSerialClient(port=port,
                             framer=ModbusRtuFramer,
                             baudrate=scanned_baudrate,
                             bytesize=8,
                             parity='N',
                             stopbits=1)

scanner.connect()

if not scanner.connected:
    print("Error connecting to modbus. Please set correct port and baudrate manually!\n")
    exit(-1)
    
for addr in range(1, 247, 1):
    response = scanner.read_holding_registers(address=0x0002,
                                             count=1,
                                             slave=addr)
    
    print(f"Scanning slave address {addr} without response\r")
    
    if response.isError():
        continue
    else:
        print(f"\nSensor might be at slave address {addr}")
        print(f"\nRegister 0x0002 contains {response.registers[0]}")
        exit(0)
        


Maybe it's possible to use this scanner and ajdust it a bit to detect at which baudrate and on which slave address the sensor operates.

In [3]:
setter = ModbusSerialClient(port=port,
                             framer=ModbusRtuFramer,
                             baudrate=scanned_baudrate,
                             bytesize=8,
                             parity='N',
                             stopbits=1)
setter.connect()

if not setter.connected:
    print("Error connecting to modbus. Please set correct port and baudrate manually!\n")
    exit(-1)
    

setter.write_register(address=0x0002,
                      value=0x05,
                      slave=scanned_address)

print("Written default device slave-address to register!")

Written default device slave-address and baudrate to register!


In [16]:
setter = ModbusSerialClient(port=port,
                             framer=ModbusRtuFramer,
                             baudrate=scanned_baudrate,
                             bytesize=8,
                             parity='N',
                             stopbits=1)
setter.connect()
if not setter.connected:
    print("Error connecting to modbus. Please set correct port and baudrate manually!\n")
    exit(-1)

setter.write_register(address=0x0003,
                      value=3,
                      slave=scanned_address)

print("Written default device baudrate to register!")

Written default device baudrate to register!
